In [ ]:
# Load truncated database into memory, ignore duplicate rows
# Import more data for better results
import sqlite3
import numpy as np
import math
from sklearn.model_selection import train_test_split
conn = sqlite3.connect('trunc_database05192019.sqlite3')
c = conn.cursor()
c.execute('SELECT DISTINCT standard_inchi,protein_sequence,standard_value FROM interactions')
d = c.fetchall()
conn.close()

In [ ]:
# Convert inchi and protein sequence into ASCII, convert IC50 into binary True/False 
din=[[0] for x in range(len(d))]
dpr=[[0] for x in range(len(d))]
dca=[[0] for x in range(len(d))]
i=0
for row in d:
    din[i] = [ord(c) for c in d[i][0]] # inchis
    dpr[i] = [ord(c) for c in d[i][1]] # proteins
    if d[i][2]=='': dca[i]=0
    elif float(d[i][2])>100: dca[i] = 0 # categories - can adjust this threshold for better results
    else: dca[i] = 1   
    i += 1

In [ ]:
# Pad with zeros to max length of each, Inchis and Proteins
# Since we concatenate the two later, need max length overall to pad so every conc row has same length
#maxlen=0
#for row in range(len(din)):
#    maxlen = len(din[row]) if len(din[row])>maxlen else maxlen
#    maxlen = len(dpr[row]) if len(dpr[row])>maxlen else maxlen # delete this if not conc later
#for row in range(len(din)):
#    din[row].extend([0 for x in range(maxlen-len(din[row]))])  
#    dpr[row].extend([0 for x in range(maxlen-len(dpr[row]))])  # delete this if not conc later
maxlen=0
for row in range(len(din)):
    maxlen = len(din[row]) if len(din[row])>maxlen else maxlen
for row in range(len(din)):
    din[row].extend([0 for x in range(maxlen-len(din[row]))])
maxlen=0
for row in range(len(dpr)):
    maxlen = len(dpr[row]) if len(dpr[row])>maxlen else maxlen
for row in range(len(dpr)):
    dpr[row].extend([0 for x in range(maxlen-len(dpr[row]))])

nin = np.array(din).astype(float) # turn into Numpy Array for Keras - INICHIS (samples x asciis)
npr = np.array(dpr).astype(float) # turn into Numpy Array for Keras - PROTEINS (samples x asciis)
nca = np.array(dca).astype(float) # turn into Numpy Array for Keras - IC50s (samples x 0/1)

In [ ]:
# Scale ASCII int to float(0..1) 
maxval = 0
for row in range(len(nin)):
    maxval = max(nin[row]) if max(nin[row])>maxval else maxval
for row in range(len(nin)):
    nin[row] = np.divide(nin[row],float(maxval))
maxval = 0
for row in range(len(npr)):
    maxval = max(npr[row]) if max(npr[row])>maxval else maxval
for row in range(len(npr)):
    npr[row] = np.divide(npr[row],float(maxval))

In [ ]:
# Import Keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.utils import np_utils
from keras.utils.vis_utils import plot_model

In [ ]:
# Create Train and Test inputs for Keras
# Concatenate Inchis and Proteins as a first try
X = np.concatenate((nin,npr),axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, nca, test_size=0.2, random_state=61)
X_train = X_train.reshape(X_train.shape[0],X_train.shape[1],1)
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1],1)
# one hot encode outputs - needed for Keras optimizer
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
num_classes = y_test.shape[1]

In [ ]:
# Build CNN with 
# 2x (1D Conv layer + MaxPooling)
# 2x Dense
# Can adjust:
# number of Conv1D layers
# number of Conv1D features (default=32, 16), size of Kernel (default=8,4)
# size of MaxPooling1D (default=2,2)
# number and size of Dense layers

model = Sequential()
model.add(Conv1D(48, (12), input_shape=(X_train.shape[1],1), activation='relu'))
model.add(MaxPooling1D(pool_size=(3)))
model.add(Conv1D(24, (8), activation='relu'))
model.add(MaxPooling1D(pool_size=(2)))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))
# Compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)
print(model.summary())

In [ ]:
batch_size = 128
epochs = 1
model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_test, y_test))

In [ ]:
# Generate probabilities for all out-of-training-sample reactions
predictions = model.predict(X_test)

In [ ]:
# Print one prediction versus actual
x=13  # change this
np.set_printoptions(precision=3)
np.set_printoptions(suppress=True)
print("Prediction Probabilities = ",predictions[x])
print("Actual Result = ",y_test[x])


In [ ]:
# Analyze each result category
for cat in range(len(predictions[0])):
    x,y = 0.0,0.0
    for rowp,rowr in zip(predictions,y_test):
        x += rowp[cat]
        y += rowr[cat]
    print("Category ",cat,": Sum(Probabilities) = ",x,"  Sum(Results) = ",y)